# Formatting into table format for kelley's algorithm

In [38]:
library(phyloseq)
library(msa)


# Autism

In [6]:
ps_og = readRDS("/oak/stanford/groups/dpwall/users/briannac/Microbiome/data/project_yogurt/phyloseq_20190909.rds")
save_dir = "/oak/stanford/groups/dpwall/users/briannac/phyloWAS/data/yogurt/tables/"

print("Reading in data...")
ps = ps_og
ps = transform_sample_counts(ps, function(x){x/sum(x)})
options(warn=-1)
ps_sub = merge_samples(ps, 'host')
options(warn=1)
sample_names(ps_sub) = sample_names(ps)[!duplicated(sample_data(ps)$host)]
sample_data(ps_sub) = sample_data(ps)[!duplicated(sample_data(ps)$host),]
ps = ps_sub

print("Cleaning up...")
x=table(paste0(sample_data(ps)$family, '_', sample_data(ps)$timepoint))
ps = subset_samples(ps, paste0(sample_data(ps)$family, '_', sample_data(ps)$timepoint) %in% names(x)[which(x==2)]) # Only keep PAIRED samples.
ps = subset_taxa(ps, colSums(otu_table(ps))>0) # Only keep PAIRED samples.
ps = transform_sample_counts(ps, function(x){x/sum(x)})
write.table(otu_table(ps), paste0(save_dir, 'person_vs_taxa.csv'), sep='\t')


[1] "Reading in data..."
[1] "Cleaning up..."


In [7]:
MSA = readRDS("/oak/stanford/groups/dpwall//users/briannac/phyloWAS/data/yogurt/msa_Muscle.rds")
save_dir = "/oak/stanford/groups/dpwall/users/briannac/phyloWAS/data/yogurt/tables/ClustalW_"
print("Dealing with MSA...")
seq_df = t(as.data.frame(sapply(as.character(MSA), function(x){strsplit(x, "")})))
rownames(seq_df) = taxa_names(ps_og)
seq_df = seq_df[taxa_names(ps),]

for (base in c('A', 'C', 'T', 'G')) {
    a = seq_df==base
    colnames(a) = sapply(1:ncol(seq_df), function(x) paste0(x, '.', base))
    if (base=='A') {df = a}
    else{df = cbind(df, a)}
}
df[df==T]=1
df[df==F]=0
write.table(seq_df, paste0(save_dir, 'taxa_vs_sequence.csv'), sep='\t')
write.table(df, paste0(save_dir, 'taxa_vs_variants.csv'), sep='\t')

[1] "Dealing with MSA..."


# Crohn's

In [43]:
# Read in and format dataframes.
for (study in c('obese_lean_twins', 'diabetes_150', 'crohns')) {
    ps = readRDS(paste0("/oak/stanford/groups/dpwall/microbiome/ncbi-datasets/", study, "/final/ps.rds"))
    save_dir = paste0("/oak/stanford/groups/dpwall/users/briannac/phyloWAS/data/" , study, "/tables/")

    # Transform sample counts and merge appropriately.
    print("Transforming Sample Counts...")
    ps = transform_sample_counts(ps, function(x){x/sum(x)})
    options(warn=-1)
    ps_sub = merge_samples(ps, 'host_subject_id')  # Change to whatever column contains host info!
    options(warn=1)
    sample_names(ps_sub) = sample_names(ps)[!duplicated(sample_data(ps)$host_subject_id)] # Change to whatever column contains host info!
    sample_data(ps_sub) = sample_data(ps)[!duplicated(sample_data(ps)$host_subject_id),] # Change to whatever column contains host info!
    ps = ps_sub
    ps = transform_sample_counts(ps, function(x){x/sum(x)})
    write.table(otu_table(ps), paste0(save_dir, 'person_vs_taxa.csv'), sep='\t')

    for (msa in c('DECIPHER', 'ClustalW', 'ClustalOmega', 'Muscle')) {
        MSA = readRDS(paste0("/oak/stanford/groups/dpwall/users/briannac/phyloWAS/data/", study, "/msa_", msa, ".rds"))
        save_dir = paste0("/oak/stanford/groups/dpwall/users/briannac/phyloWAS/data/", study, "/tables/",  msa, "_")
        message("Dealing with MSA...")
        seq_df = t(as.data.frame(sapply(as.character(MSA), function(x){strsplit(x, "")})))
        rownames(seq_df) = taxa_names(ps)
        seq_df = seq_df[taxa_names(ps),]
        consensus_mat = consensusMatrix(MSA)
        consensus_seq = apply(consensus_mat, 2, function(x) rownames(consensus_mat)[which.max(x)])
        isConsensus = t(apply(seq_df, 1, function(x) (x==consensus_seq)))
        seq_df[isConsensus] = 'X'
        for (base in c('A', 'C', 'T', 'G')) {
            a = seq_df==base
            colnames(a) = sapply(1:ncol(seq_df), function(x) paste0(x, '.', base))
            if (base=='A') {df = a}
            else{df = cbind(df, a)}
        }
        df[df==T]=1
        df[df==F]=0
        write.table(seq_df, paste0(save_dir, 'taxa_vs_sequence.csv'), sep='\t')
        write.table(df, paste0(save_dir, 'taxa_vs_variants.csv'), sep='\t')
    }
}

[1] "Transforming Sample Counts..."


Dealing with MSA...
Dealing with MSA...
Dealing with MSA...
Dealing with MSA...


[1] "Transforming Sample Counts..."


Dealing with MSA...
Dealing with MSA...
Dealing with MSA...
Dealing with MSA...


[1] "Transforming Sample Counts..."


Dealing with MSA...
Dealing with MSA...
Dealing with MSA...
Dealing with MSA...


In [46]:
ps = readRDS("/oak/stanford/groups/dpwall/microbiome/ncbi-datasets/diabetes_150/final/ps.rds")


In [55]:
ps = readRDS("/oak/stanford/groups/dpwall/microbiome/ncbi-datasets/crohns/final/ps.rds")
sample_data(ps)$diseasesubtype=='UC'

[1] M1-0128 M1-0632 M1-0325 M1-0438 M1-0201 M1-0297 M1-0142 M1-0020 M1-0284
 [10] M1-0383 M1-0111 M1-0462 M1-0161 M1-0018 M1-0679 M1-0141 M1-0346 M1-0279
 [19] M1-0463 M1-0354 M1-0633 M1-0435 M1-0260 M1-0738 M1-0317 M1-0336 M1-0054
 [28] M1-0135 M1-0668 M1-0545 M1-0635 M1-0109 M1-0283 M1-0736 M1-0262 M1-0032
 [37] M1-0689 M1-0629 M1-0391 M1-0203 M1-0160 M1-0464 M1-0337 M1-0648 M1-0605
 [46] M1-0540 M1-0102 M1-0103 M1-0396 M1-0246 M1-0387 M1-0844 M1-0511 M1-0637
 [55] M1-0670 M1-0332 M1-0764 M1-0763 M1-0234 M1-0296 M1-0467 M1-0196 M1-0216
 [64] M1-0025 M1-0739 M1-0377 M1-0012 M1-0746 M1-0528 M1-0489 M1-0028 M1-0481
 [73] M1-0389 M1-0650 M1-0253 M1-0255 M1-0254 M1-0006 M1-0623 M1-0639 M1-0252
 [82] M1-0382 M1-0440 M1-0598 M1-0077 M1-0667 M1-0329 M1-0765 M1-0035 M1-0362
 [91] M1-0663 M1-0494 M1-0399 M1-0784 M1-0328 M1-0183 M1-0653 M1-0795 M1-0507
[100] M1-0755 M1-0301 M1-0767 M1-0063 M1-0298 M1-0380 M1-0381 M1-0251 M1-0522
[109] M1-0661 M1-0009 M1-0333 M1-0331 M1-0701 M1-0603 M1-0385 M1-0468 M1-0640
[118] M1-0687 M1-0780 M1-0482 M1-0786 M1-0695 M1-0007 M1-0434 M1-0552 M1-0219
[127] M1-0728 M1-0114 M1-0172 M1-0112 M1-0602 M1-0802 M1-0131 M1-0535 M1-0684
[136] M1-0788 M1-0151 M1-0486 M1-0666 M1-0686 M1-0413 M1-0051 M1-0133 M1-0649
[145] M1-0665 M1-0756 M1-0744 M1-0411 M1-0400 M1-0014 M1-0423 M1-0551 M1-0249
[154] M1-0037 M1-0245 M1-0456 M1-0588 M1-0024 M1-0237 M1-0073 M1-0226 M1-0225
[163] M1-0669 M1-0495 M1-0461 M1-0125 M1-0734 M1-0497 M1-0769 M1-0084 M1-0319
[172] M1-0457 M1-0628 M1-0590 M1-0601 M1-0373 M1-0322 M1-0269 M1-0384 M1-0351
[181] M1-0737 M1-0068 M1-0008 M1-0688 M1-0320 M1-0403 M1-0549 M1-0700 M1-0596
[190] M1-0800 M1-0818 M1-0023
192 Levels: M1-0006 M1-0007 M1-0008 M1-0009 M1-0012 M1-0014 M1-0018 ... M1-0844